# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

1. [Загрузка и подготовка данных](#shag_1)
2. [Обучение и проверка модели](#shag_2)
3. [Подготовка к расчету прибыли](#shag_3)
4. [Расчет прибыли и рисков](#shag_4)

<a id='shag_1'></a>

# 1. Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from statistics import mean
from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, accuracy_score, mean_squared_error

В этом разделе будем поочереди открывать пулы данных.

In [2]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')
display(data0)
display(data1)
display(data2)
print(data0.info())
print(data1.info())
print(data2.info())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

У нас есть 3 файла с данными - по файлу для каждого региона добычи. В каждом файле id скважины, 3 фактора и объем добычи. Пропущенных значений нет. Создадим функцию для выделения выборки из датафрейма. Поскольку в одном регионе может быть главным один фактор, а в другом регионе - другой, будем делать отдельные выборки по регионам и проводить отдельное обучение.

In [3]:
def features_target(data):
    features = data.drop(['product','id'],1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345) 

    return(features_train,target_train,features_valid,target_valid)

features_train,target_train,features_valid,target_valid = features_target(data0)

<a id='shag_2'></a>

# 2. Обучение и проверка модели

Воспользуемся нашей функцией для разбиения выборки для 1 региона.
Обучим модель и сделаем предсказания на валидационной выборке.
Сохраним предсказания и правильные ответы на валидационной выборке.
Посмотрим на средний запас предсказанного сырья и RMSE модели.

In [4]:
features_train0,target_train0,features_valid0,target_valid0 = features_target(data0)
model = LinearRegression().fit(features_train0, target_train0)
target_predict0 = model.predict(features_valid0)
target_predict0_valid=pd.DataFrame(target_predict0, index=target_valid0.index)
target_predict0_valid.columns=['predict']
target_predict0_valid['valid']=target_valid0
print(target_predict0_valid)
#r_sq = model.score(features_train, target_train)
#mean_product = mean(target_predict)
#RMSE = mean_squared_error(target_valid,target_predict)

          predict       valid
71751   95.894952   10.038645
80493   77.572583  114.551489
2655    77.892640  132.603635
53233   90.175134  169.072125
91141   70.510088  122.325180
...           ...         ...
12581  103.037104  170.116726
18456   85.403255   93.632175
73035   61.509833  127.352259
63834  118.180397   99.782700
43558  118.169392  177.821022

[25000 rows x 2 columns]


In [5]:
def print_result_model(target_predict,target_valid,features_train,target_train,):
    #r_sq = model.score(features_train, target_train)
    mean_product = mean(target_predict)
    sum_product=target_predict.sum()
    RMSE = sqrt(mean_squared_error(target_valid,target_predict))
    #print('coefficient of determination:', r_sq)
    #print(model.intercept_, model.coef_)
    print('RMSE:', RMSE)
    print('Средний объем запасов:', mean_product)
    return(sum_product)

In [6]:
print_result_model(target_predict0,target_valid0,features_train0,target_train0)

RMSE: 37.5794217150813
Средний объем запасов: 92.59256778438038


2314814.1946095093

По первому региону получили данные

RMSE: 37.5794217150813

Средний объем запасов: 92.59256778438038

In [7]:
features_train1,target_train1,features_valid1,target_valid1 = features_target(data1)
model = LinearRegression().fit(features_train1, target_train1)
target_predict1 = model.predict(features_valid1)
target_predict1_valid=pd.DataFrame(target_predict1, index=target_valid1.index)
target_predict1_valid.columns=['predict']
target_predict1_valid['valid']=target_valid1
print_result_model(target_predict1,target_valid1,features_train1,target_train1,)

RMSE: 0.893099286775616
Средний объем запасов: 68.72854689544602


1718213.67238615

По второму региону получили 

RMSE: 0.893099286775616

Средний объем запасов: 68.72854689544602

In [8]:
features_train2,target_train2,features_valid2,target_valid2 = features_target(data2)
model = LinearRegression().fit(features_train2, target_train2)
target_predict2 = model.predict(features_valid2)
target_predict2_valid=pd.DataFrame(target_predict2, index=target_valid2.index)
target_predict2_valid.columns=['predict']
target_predict2_valid['valid']=target_valid2
print_result_model(target_predict2,target_valid2,features_train2,target_train2,)
print_result_model(target_predict2,target_valid2,features_train2,target_train2,)

RMSE: 40.02970873393434
Средний объем запасов: 94.9650459680049
RMSE: 40.02970873393434
Средний объем запасов: 94.9650459680049


2374126.1492001223

По третьему региону получили 

RMSE: 40.02970873393434

Средний объем запасов: 94.9650459680049

В третьем регионе самый высокий предсказанный объем сырья, но и самая высокая среднеквадратичная ошибка.

<div class="alert alert-info">  Добавил корень в RMSE </div>

<a id='shag_3'></a>

# 3. Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраним в отдельных переменных.
Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
Сделаем выводы по этапу подготовки расчёта прибыли.

Затраты на каждый регион на 200 скважин составляют 10 000 000 тыс. руб. Выручка будет равна объему с 200 лучших скважин умноженная на 450 тыс. руб. Итого полученная прибыль = выручка - затраты = запасы*450-10000000 (в тыс. руб.)

In [9]:
print('Безубыточный объем добьем добычи будет составлять:',round(10000000/200/450,2),'тыс.баррелей')

Безубыточный объем добьем добычи будет составлять: 111.11 тыс.баррелей


Видим что показатель безубыточного объема превышает расчитанные средний объемы во всех регионах.

In [10]:
def profit_calc(target_valid_sub):
    price = 450
    cost = 10000000
    sum_profit=target_valid_sub['valid'].sum()*price-cost
    return(sum_profit)  

In [11]:
def predict_zapas(target_predict):
    predict_zapas0=pd.DataFrame(target_predict)
    predict_zapas0.columns=['product']
    predict_zapas0=predict_zapas0.sort_values(by='product',ascending=False)[:200]
    return(predict_zapas0)

In [12]:
def profit_region(data):
    data['income']=round(data['target_predict']*450-50000)
    data['profit']=((data['income']>=0)*1)
    return(data)

В этой функции создаем столбец выручки ['income'] со скважины в тыс.руб. и столбец ['profit'], показывающий превышает ли выручка скважины ее себестоимость.

Риск убыточности скважины расчитаем как отношение всех скважин с объемом добычи менее 111.11 тыс.бар к общему их числу

In [27]:
def profit_target_by_predict(target_predict, target_valid):
    target_predict_sorted=[]
    target_predict = pd.Series(target_predict)   
    #result = target_valid_selected.sum() * barel - budget  
    total_target = target_predict.to_frame().join(target_valid.to_frame(),how='inner')
    total_target.columns=['target_predict', 'target_valid']
    total_target = total_target.sort_values(by=['target_predict'],ascending=False)[:200]
    return(total_target)

def profit_1(target, predictions):
    predictions = pd.Series(predictions)
    pred_sorted = predictions.sort_values(ascending=True)
    select_points = target[pred_sorted.index][:200]
    product = select_points.sum()
    rev = product * 450
    return rev - 10000000

state = np.random.RandomState(12345)
def profit(pred_valid, target_valid, n):
    predictions_valid_sorted = pd.Series(pred_valid).sort_values(ascending=False) #.reset_index(drop=True)
    target_valid_selected = target_valid[predictions_valid_sorted.index][:n]
    result = target_valid_selected.sum() * 450 - 10000000
#     print('Прибыль с 200 лучших скважин составила - {:.2f} тысяч рублей'.format(result))
    return result

def bootstrap_calc(target_predict_valid):
    risk_list = []
    profit_list=[]
    for i in range(1000):
        target_valid_sub = target_predict_valid.sample(n=500, replace=True, random_state=state) #.reset_index(drop=True) #Тут у нас 500 значений
        target_valid_sub = target_valid_sub.sort_values(by=['predict'],ascending=False)[:200]
        target_valid_sub['unprofit']=((target_valid_sub['valid']<111.11 )*1) 
        risk = target_valid_sub['unprofit'].sum()/200
        risk_list.append(risk)
        total_profit = profit_calc(target_valid_sub)
        profit_list.append(total_profit)
    profit_series = pd.Series(profit_list)
    risk_series = pd.Series(risk_list)
    mean_profit = profit_series.mean()
    confidence_interval = (profit_series.quantile(0.025), profit_series.quantile(0.975))
    risk = (((profit_series < 0).sum())/1000)*100
    #print('риск убыточной скважины = ',risk,'%')
    print('риск убыточной скважины = ',risk,'%')    # ДОБАВЛЕНО РЕВЬЮЕРОМ
    print()
    print('Средняя прибыль рибыль =',mean_profit,'тыс.руб.')
    print()
    print("95%-ый доверительный интервал:", confidence_interval)
    return(risk,mean_profit,confidence_interval)


<a id='shag_4'></a>

# 4. Расчёт прибыли и рисков 

Выберем скважины с максимальными значениями предсказаний.
Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
Рассчитаем прибыль для полученного объёма сырья.

In [14]:
region_0 = profit_target_by_predict(target_predict0, target_valid0)
region_0 = profit_region(region_0)
print('Прибыль 1 региона = ',region_0['income'].sum(), 'тыс.руб.')

Прибыль 1 региона =  2912877.0 тыс.руб.


In [15]:
region_1 = profit_target_by_predict(target_predict1, target_valid1)
region_1 = profit_region(region_1)
print('Прибыль 2 региона = ',region_1['income'].sum(), 'тыс.руб.')

Прибыль 2 региона =  2431596.0 тыс.руб.


In [16]:
region_2 = profit_target_by_predict(target_predict2, target_valid2)
region_2 = profit_region(region_2)
print('Прибыль 3 региона = ',region_2['income'].sum(), 'тыс.руб.')

Прибыль 3 региона =  2496013.0 тыс.руб.


Самая высокая прибыль в 3 регионе.

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
Сделаем выводы: какой регион подходит для разработки скважин и почему.

Расчет для первого региона:

In [28]:
df = pd.DataFrame(columns = ['Риск', 'Средняя прибыль', 'Интервал'])
risk,mean_profit,confidence_interval = bootstrap_calc(target_predict0_valid)
df.loc[0] = [risk,mean_profit,confidence_interval]

риск убыточной скважины =  6.9 %

Средняя прибыль рибыль = 396164.98480237107 тыс.руб.

95%-ый доверительный интервал: (-111215.54589049605, 909766.941553422)


Расчет для второго региона:

In [29]:
risk,mean_profit,confidence_interval = bootstrap_calc(target_predict1_valid)
df.loc[1] = [risk,mean_profit,confidence_interval]

риск убыточной скважины =  0.7000000000000001 %

Средняя прибыль рибыль = 461155.8172772397 тыс.руб.

95%-ый доверительный интервал: (78050.81075174107, 862952.0602637231)


Расчет для третьего региона:

In [30]:
risk,mean_profit,confidence_interval = bootstrap_calc(target_predict2_valid)
df.loc[2] = [risk,mean_profit,confidence_interval]

риск убыточной скважины =  6.5 %

Средняя прибыль рибыль = 392950.4751706045 тыс.руб.

95%-ый доверительный интервал: (-112227.62537857569, 934562.914551164)


In [31]:
display(df)

,Риск,Средняя прибыль,Интервал
0,6.9,396164.984802,"(-111215.54589049605, 909766.941553422)"
1,0.7,461155.817277,"(78050.81075174107, 862952.0602637231)"
2,6.5,392950.475171,"(-112227.62537857569, 934562.914551164)"


В результате расчетов выяснили, что доверительный интервал прибыли и расчитанная средняя прибыль по второму региону добычи самая высокая, риск убыика практически отсутствует (0,7%). К разработке новых скважин рекомендован 2 регион.

*********************

In [21]:
# функция прибыли 
def profit_1(target, predictions):
    pred_sorted = predictions.sort_values(ascending=False)
    select_points = target[pred_sorted.index][:200]
    product = select_points.sum()
    rev = product * PRODUCT_PRICE
    return rev - cost

*******************

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован